In [44]:
from pyspark import SparkConf, SparkContext, RDD
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import col,split,countDistinct,min
from pyspark.streaming import StreamingContext




In [2]:
import pyspark

In [2]:
import pandas as pd





In [3]:
def row_flatmap(x):
    elements = x.split(",")
    attr = elements[1].split(';')
    num = elements[2].split(';')
    l = []

    for i in range(0,5):
        l.append(elements[0] + ',' + attr[i] + ',' + num[i])

    
    return l
        
        
   
    

In [4]:
def get_spark_context(on_server) -> SparkContext:
    spark_conf = SparkConf().setAppName("2ID70-MS2")
    if not on_server:
        spark_conf = spark_conf.setMaster("local[*]")
    return SparkContext.getOrCreate(spark_conf)


In [4]:
def q1(spark_context: SparkContext, on_server) -> RDD:
    database_file_path = "/Database.csv" if on_server else "Database.csv"

    # TODO: You may change the value for the minPartitions parameter (template value 160) when running locally.
    # It is advised (but not compulsory) to set the value to 160 when running on the server.
    database_rdd = spark_context.textFile(database_file_path, 160)
    
    # TODO: Implement Q1 here by defining q1RDD based on databaseRDD.
    q1_rdd = None

    return q1_rdd

In [5]:
conf = SparkConf().setAppName("2ID70-MS2")
sc = SparkContext(conf=conf)


In [6]:
database_rdd = sc.textFile("C:/Users/20190938/Desktop/m2/Database.csv", 160)

In [7]:
header = database_rdd.first()
q1_rdd = database_rdd\
    .filter(lambda x: x != header)

In [8]:
q1_rdd = q1_rdd.flatMap(lambda x: row_flatmap(x))

In [9]:
q1_rdd.take(10)

['A,c,19619',
 'A,q,33917',
 'A,s,0',
 'A,u,117',
 'A,z,48337',
 'A,c,31260',
 'A,q,28358',
 'A,s,11',
 'A,u,43',
 'A,z,22051']

In [14]:
q1s_rdd = q1_rdd.filter(lambda x: x[0] == 'S')


In [26]:
s = q1s_rdd.count()

In [19]:
q1t_rdd = q1_rdd.filter(lambda x: x[0] == 'T')

t = q1t_rdd.count()


In [22]:
t

250000

In [23]:
q1r_rdd = q1_rdd.filter(lambda x: x[0] == 'R')

r =  q1r_rdd.count()

In [28]:
print(">> [q1: R: " + str(r) + "]")
print(">> [q1: S: " + str(s) + "]")
print(">> [q1: T: " + str(t) + "]")

>> [q1: R: 250000]
>> [q1: S: 250000]
>> [q1: T: 250000]


In [31]:
def q2(spark_context: SparkContext, q1_rdd: RDD):
    spark_session = SparkSession(spark_context)

    # TODO: Implement Q2 here.




In [10]:
sc1 = SparkContext.getOrCreate(conf)

spark_session = SparkSession(sc1)

In [11]:
data_frame = q1_rdd.map(lambda r:Row(r)).toDF(["RelationName"])

In [13]:
data_frame = data_frame.select(split(data_frame.RelationName,",")).rdd.flatMap(lambda x: x).toDF(schema =['rel','name','no'])

AttributeError: 'DataFrame' object has no attribute 'RelationName'

In [16]:
data_frame.filter(col("rel") == 'R').count()

250000

In [33]:
q22 = data_frame.groupBy(["name"]).agg(countDistinct("no"))


In [37]:
q23 = q22.filter(col("count(no)")>=1000)

In [ ]:
q23.count()

In [48]:
q22.agg(min(col("count(no)"))).take(1)

[Row(min(count(no))=493)]

In [ ]:
def q3(spark_context: SparkContext, q1_rdd: RDD):
    # TODO: Implement Q3 here.







In [ ]:
a

In [ ]:
def q4(spark_context: SparkContext, on_server):
    streaming_context = StreamingContext(spark_context, 2)
    streaming_context.checkpoint("checkpoint")

    hostname = "stream-host" if on_server else "localhost"
    lines = streaming_context.socketTextStream(hostname, 9000)

    # TODO: Implement Q4 here.

    # Start the streaming context, run it for two minutes or until termination
    streaming_context.start()
    streaming_context.awaitTerminationOrTimeout(2 * 60)
    streaming_context.stop()

In [ ]:
# Main 'function' which initializes a Spark context and runs the code for each question.
# To skip executing a question while developing a solution, simply comment out the corresponding function call.
if __name__ == '__main__':

    on_server = False  # TODO: Set this to true if and only if running on the server

    spark_context = get_spark_context(on_server)

    q1_rdd = q1(spark_context, on_server)

    q2(spark_context, q1_rdd)

    q3(spark_context, q1_rdd)

    q4(spark_context, on_server)

    spark_context.stop()